In [9]:
import random
import numpy as np
import time
import os
import json
import copy

status = []  # 存放n皇后坐标，index i表示row，对应元素值表示col，且元素值互不相同，从0到n-1
d1 = []  # 存放每条正对角线上的皇后个数，长度2n-1
d2 = []  # 存放每条负对角线上的皇后个数，长度2n-1
d2_base = 0  # 负对角线row-col范围为[-(2n-1)/2,(2n-1)/2]，转化成index需要加上base = (2n-1)/2
collisions = 0  # 冲突数

# 更新冲突数，将d1和d2中大于1的元素减1后累加,时间复杂度O(n)
def update_collisions():
    global collisions
    collisions = 0
    for i in d1:
        if i>1:
            collisions += i-1
    for j in d2:
        if j>1:
            collisions += j-1

# 判断row行上的皇后是否处在有冲突的位置，即d1和d2对应元素值是否大于1
def is_attacked(row):
    if d1[row+status[row]]>1:
        return True
    elif d2[d2_base+row-status[row]]>1:
        return True
    
    return False

# 测试调换皇后之后的collisions的变化
# 由于每个皇后只会影响两条对角线，那么两个皇后对换最多可能影响8条对角线，即d1和d2中的8个元素。
# 只需将可能影响到的元素取出而不需要遍历，时间复杂度O(1)
def swap_test(i,j):
    collisions_change = 0
    
    # 两个“源”皇后对应4条对角线上的值
    d1_i_s = d1[i+status[i]]
    d2_i_s = d2[d2_base+i-status[i]]
    d1_j_s = d1[j+status[j]]
    d2_j_s = d2[d2_base+j-status[j]]
    # 两个“目的”皇后对应4条对角线上的值
    d1_i_d = d1[i+status[j]]
    d2_i_d = d2[d2_base+i-status[j]]
    d1_j_d = d1[j+status[i]]
    d2_j_d = d2[d2_base+j-status[i]]
    
    # 取走两个“源”皇后最多会影响4条对角线
    if d1_i_s >1:
        collisions_change -= 1
    if d2_i_s >1:
        collisions_change -= 1
    if d1_j_s >1:
        collisions_change -= 1
    if d2_j_s >1:
        collisions_change -= 1
    # 有可能两个皇后在同一条正/负对角线上，此时若对应值为2，则change值多减了1，需要加1
    if i+status[i] == j+status[j] and d1_i_s == 2:
        collisions_change += 1
    if i-status[i] == j-status[j] and d2_i_s == 2:
        collisions_change += 1
    
    # 放置两个“目的”皇后最多会影响4条对角线
    if d1_i_d >=1:
        collisions_change += 1
    if d2_i_d >=1:
        collisions_change += 1
    if d1_j_d >=1:
        collisions_change += 1
    if d2_j_d >=1:
        collisions_change += 1
    # 同样，有可能两个皇后在同一条正/负对角线上，此时若对应值为0，则change值少加了1，需要回复加1
    if i+status[j] == j+status[i] and d1_i_d == 0:
        collisions_change += 1
    if i-status[j] == j-status[i] and d2_i_d == 0:
        collisions_change += 1
        
    return collisions_change

# 实行调换动作，更新d1,d2
def swap_performed(i,j):
    # 更新d1,d2
    d1[i+status[i]]-=1
    d2[d2_base+i-status[i]]-=1
    d1[j+status[j]]-=1
    d2[d2_base+j-status[j]]-=1
    d1[i+status[j]]+=1
    d2[d2_base+i-status[j]]+=1
    d1[j+status[i]]+=1
    d2[d2_base+j-status[i]]+=1
    # 对换位置
    temp = status[i]
    status[i] = status[j]
    status[j] = temp

def local_search():
    global collisions
    length = len(status)
       
    # repeat
    # for all i, j: where 皇后 status[i] 或 status[j] is attacked do
        # If 交换 status[i] 与 status[j] 能减少冲突
            # Then 交换 status[i] 与 status[j]
    # until collisions = 0 或 卡在局部最优
    iter_num = 0
    while collisions >0 and iter_num <5:
        for row_i in range(length):
            for row_j in range(row_i,length):
                if is_attacked(row_i) or is_attacked(row_j):
                    collisions_change = swap_test(row_i,row_j)
                    if collisions_change < 0:
                        swap_performed(row_i,row_j)
                        # print(status)
                        collisions += collisions_change
        iter_num += 1
    update_collisions()
    print("最终冲突数："+str(collisions)+'\n')
                    
    return status

# 初始化
# 保证前n-c行的皇后互不冲突，后c行随机放置
# candidate=shuffle([0,1,...,n])确保每一行/列上不会有冲突的皇后对
def initialize(size,c):
    
    candidate = []
    for i in range(size):
        candidate.append(i)
    random.shuffle(candidate)  # 打乱数组顺序
    
    # 初始化d1,d2,collision
    global status,d1,d2,d2_base
    status = []
    d2_base = (2*size - 1)//2
    d1 = [0]*(2*size-1)
    d2 = [0]*(2*size-1)
    
    # 前n-c行的皇后互不冲突
    for row in range(0,size-c):
        for col in candidate:
            if d1[row+col] == 0 and d2[d2_base+row-col] == 0:
                status.append(col)
                candidate.remove(col)
    # 后c行的皇后在与之前的皇后不同列的情况下随机放置
    for row in range(size-c,size):
        for col in candidate:
            status.append(col)
    
    
    print("棋盘初始状态：")
    print(status)
    print("棋盘初始冲突数：")
    print(str(collisions) + '\n')
    
    return status
                
# 求得满足冲突数为0的一个解
def Queens_Search4(size, c, filename):
    
    # 计时
    time_start = time.time()
    # 初始化
    status = initialize(size,c)
    # 进行局部搜索
    status = local_search()
    # 当一个随机解不能优化且未能消除冲突时，算法将重新生成一个随机解，并进行局部搜索
    cnt = 1
    while collisions>0 :
        print("卡在局部最优，重新初始化")
        print("重新初始化次数: ",cnt)
        status = initialize(size,c)    # 重新初始化
        status = local_search()
        cnt += 1
        
    time_end = time.time()
    time_cost = time_end - time_start
    
    print("棋盘最终状态：")
    print(status)
    
    # 将结果写入文件并输出
    h = open(filename, 'w', encoding='utf-8')
    h.write('问题规模：' + str(size) + '\n')
    h.write('搜索时间：' + str(time_cost) + '\n')
    h.write('重新初始化次数：' + str(cnt) + '\n')
    h.write('解：' + str(status) + '\n')
    h.close()

In [12]:
dir = "qs4_config.json"
while not os.path.exists(dir):
# 判断文件是否存在
    print("file doesn't exists!")
    sys.exit()
f = open(dir, encoding='utf-8')
setting = json.load(f)
f.close()

num = setting['num']
c = setting['c']
outputfile = setting['output']

Queens_Search4(num,c,outputfile)

棋盘初始状态：
[88568, 50509, 76195, 86908, 90372, 51195, 65436, 43286, 65707, 13808, 46670, 48540, 65860, 42029, 7397, 80231, 89164, 66969, 41320, 56962, 71364, 94810, 74154, 63813, 50020, 39680, 26044, 67426, 3866, 24894, 77815, 95216, 62741, 59370, 19526, 14354, 2159, 63703, 55306, 58173, 78836, 39150, 41785, 87799, 58507, 29935, 39231, 15710, 22982, 95060, 63213, 23651, 83899, 32268, 76941, 49965, 74057, 98287, 37997, 9680, 60601, 44401, 51148, 85979, 41463, 33780, 4904, 86128, 82793, 57940, 5961, 36002, 35378, 94842, 74220, 74842, 65776, 36992, 86784, 87643, 99208, 76436, 14892, 92730, 99817, 92882, 87324, 3698, 43792, 59596, 66588, 56586, 34125, 56421, 94133, 54109, 58482, 63830, 16270, 9201, 45210, 85023, 47602, 87921, 76615, 12720, 81709, 42853, 6569, 11563, 87492, 21390, 81120, 27423, 55464, 42360, 48902, 25270, 73227, 93426, 11857, 67753, 32405, 31433, 98111, 71200, 49708, 20192, 19712, 80169, 92416, 23933, 16023, 24996, 33414, 20173, 92787, 9743, 56205, 83418, 42628, 38535, 56758, 